<a href="https://colab.research.google.com/github/AnaJlia999/gsi073/blob/main/GSI073_aula0_support_vector_machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import datasets
from sklearn.model_selection import train_test_split

# Preparar o dataset
iris = datasets.load_iris()
X = iris.data; y = iris.target

X = X[y != 1] ; y = y[y != 1]  # Versicolor
y = torch.tensor(y, dtype=torch.float32)
y[y == 0] = -1  # SVM espera rótulos em {-1, +1}

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)

# Usar nn.Linear para substituir w e b
n_features = X_train.shape[1]
modelo = nn.Linear(n_features, 1)  # Camada linear que já tem os parâmetros w e b internos

# Hiperparâmetros
learning_rate = 0.01
epochs = 300
optimizer = optim.Adam(modelo.parameters(), lr=learning_rate)

# Execução do treinamento
for epoch in range(epochs):
    optimizer.zero_grad()

    # Passar os dados pela camada linear (equivalente ao cálculo X * w + b)
    y_pred = modelo(X_train)

    # Hinge loss
    perda_de_classificacao = torch.clamp(1 - y_train.view(-1, 1) * y_pred, min=0).mean()

    # Regularização L2
    perda_de_distancia_entre_classes = 0.5 * torch.sum(modelo.weight ** 2)  # Regularização nos pesos (w)

    # Função objetivo
    loss = perda_de_distancia_entre_classes + perda_de_classificacao

    loss.backward()  # Calcula os gradientes
    optimizer.step()  # Atualiza os parâmetros

    if (epoch + 1) % 100 == 0:
        print(f"Epoch {epoch+1}/{epochs}, Loss={loss.item():.4f}")

Epoch 100/300, Loss=0.2572
Epoch 200/300, Loss=0.1200
Epoch 300/300, Loss=0.1069


In [32]:
# Avaliação no conjunto de teste
with torch.no_grad():
    y_pred = modelo(X_test)  # Predição do modelo
    y_pred_class = torch.sign(y_pred)  # Classifica as predições (-1 ou +1)
    acuracia = (y_pred_class.view(-1) == y_test).float().mean()  # Calcula a acurácia
    print(f'Acurácia no conjunto de teste: {acuracia.item():.4f}')

Acurácia no conjunto de teste: 0.6000
